# Singapore housing data extraction

## Author: Ankur Shanker

## Student ID: 21159916

### **Section 1:** Importing packages necessary for workbook exectuion.

Libraries necessary for workbook execution related to functionality responsible for geo-spatial data reading, plotting, and statistical analysis need to be imported in order for this workbook to function.

In [1]:
# Import packages required for analysis
import pandas as pd
from sklearn.model_selection import train_test_split
import rfpimp
from sklearn import model_selection

# Import packages required for numeric operations
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
import statsmodels.api as sm

In [2]:
# Import packages required for geospatial analysis
import geopandas as gpd
import pyproj
import geopy.distance
pyproj.datadir.get_data_dir()

'c:\\Users\\ankur\\anaconda3\\lib\\site-packages\\pyproj\\proj_dir\\share\\proj'

In [3]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

In [4]:
# Import packages required for data visualisation
import matplotlib.pyplot as plt
import seaborn as sn
sn.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})

### **Section 2:** Reading and formatting data

Data related to house resale prices in Singapore is read in below.

In [5]:
singapore_2000_onwards = pd.read_csv('Data/Singapore data/Singapore 2000 onwards.csv')
singapore_2000_onwards = gpd.GeoDataFrame(singapore_2000_onwards.loc[:, [c for c in singapore_2000_onwards.columns if c != "geometry"]], geometry=gpd.GeoSeries.from_wkt(singapore_2000_onwards["geometry"]))
singapore_2000_onwards

,ID,floor_area_sqm,mean_resale_price,Closest distance to Marina Bay Sands,Closest distance to Resorts World Casino Sentosa,geometry
0,block 1 BEDOK,87.151515,338909.623855,9.057217e+03,9.057217e+03,POINT (103.93147 1.32567)
1,block 1 BUKIT MERAH,84.145833,417188.365358,2.915024e+03,2.915024e+03,POINT (103.83535 1.27935)
2,block 1 BUKIT TIMAH,117.789474,698816.998405,2.635760e+05,2.635760e+05,POINT (101.46708 1.51128)
3,block 1 CENTRAL AREA,62.666667,419808.253968,6.335036e+06,6.335036e+06,POINT (146.00830 -41.10814)
4,block 1 GEYLANG,82.290323,428180.333312,4.703110e+03,4.703110e+03,POINT (103.88706 1.31819)
...,...,...,...,...,...,...
1967,block 944 TAMPINES,145.727273,684870.708333,1.131450e+04,1.131450e+04,POINT (103.93169 1.35824)
1968,block 946 HOUGANG,112.750000,422688.676740,7.787478e+03,7.787478e+03,POINT (103.88324 1.35147)
1969,block 950 HOUGANG,110.875000,432444.543651,7.787478e+03,7.787478e+03,POINT (103.88324 1.35147)
1970,block 954 HOUGANG,112.111111,423138.606934,7.787478e+03,7.787478e+03,POINT (103.88324 1.35147)


Data related to singapore's hotel room rates and geolocations is also read in, as only distances to hotels are being considered the hotel room rate data is disregarded.

In [6]:
singapore_hotels = pd.read_csv('Data/Singapore Data/Singapore hotels/Singapore hotels.csv')
singapore_hotels = singapore_hotels[['Name','geometry']]
singapore_hotels = gpd.GeoDataFrame(singapore_hotels.loc[:, [c for c in singapore_hotels.columns if c != "geometry"]], geometry=gpd.GeoSeries.from_wkt(singapore_hotels["geometry"]))
singapore_hotels

,Name,geometry
0,"Hotel Boss, Singapore",POINT (103.86010 1.30579)
1,YOTELAIR Singapore Changi Airport,POINT (103.99053 1.35988)
2,"PARKROYAL COLLECTION Marina Bay, Singapore",POINT (103.85682 1.29161)
3,"The Fullerton Hotel, Singapore\t352",POINT (103.85307 1.28620)
4,"Carlton Hotel, Singapore",POINT (103.85259 1.29572)
...,...,...
175,"Betel Box Backpackers Hostel, Singapore",POINT (103.90027 1.31227)
176,"Spacepod, Kallang, Singapore",POINT (103.86193 1.31052)
177,"The Bohemian, Chinatown, Singapore",POINT (103.84492 1.28382)
178,"Atelier, Chinatown, Singapore",POINT (103.84677 1.28069)


In [8]:
#https://data.gov.sg/dataset/master-plan-2014-subzone-boundary-web?resource_id=1c6b586b-61ca-45a9-b704-df4c9057fbd6

### **Section 3:** Point of interest proximity and concentration

In order to complement the core regression analysis of this workbook, which relies on determining the relationship between casino proxmity and house prices in Singapore, externalities related to the concentrations of geospatial attributes are also considered.

Consequently, the distance from, and concentration of these geospatial externalities with repsect to the location of centroids of Singapore's residential housing blocks can be determined via the utilisation of a method that determines the point of interest count up until a certain radius around a hotel (determinePOICount) and the use of a method that can calculate the distance of an external feature from a hotel (CalculateDistance).

In [17]:
def CalculateDistance(attribute, tag):
    distances = [];
    for i, row in singapore_2000_onwards.iterrows():

        closest_distance= 1000000000000000

        for j, location in attribute.iterrows():
            coords1 = (singapore_2000_onwards.at[i, 'geometry'].y, singapore_2000_onwards.at[i, 'geometry'].x)
            coords2 = (attribute.at[j, 'geometry'].y, attribute.at[j, 'geometry'].x)
            distance = geopy.distance.geodesic(coords1, coords2).m    
            if(distance < closest_distance):
                closest_distance = distance
        distances.append(closest_distance)
    singapore_2000_onwards[tag] = distances

The distance of a housing block centroid to a it's closest hotel is added to the dataframe below.

In [18]:
CalculateDistance(singapore_hotels, 'Closest distance to a hotel')
singapore_2000_onwards

,ID,floor_area_sqm,mean_resale_price,Closest distance to Marina Bay Sands,Closest distance to Resorts World Casino Sentosa,geometry,Closest distance to a hotel
0,block 1 BEDOK,87.151515,338909.623855,9.057217e+03,9.057217e+03,POINT (103.93147 1.32567),1.853188e+03
1,block 1 BUKIT MERAH,84.145833,417188.365358,2.915024e+03,2.915024e+03,POINT (103.83535 1.27935),4.987472e+02
2,block 1 BUKIT TIMAH,117.789474,698816.998405,2.635760e+05,2.635760e+05,POINT (101.46708 1.51128),2.536406e+05
3,block 1 CENTRAL AREA,62.666667,419808.253968,6.335036e+06,6.335036e+06,POINT (146.00830 -41.10814),6.332915e+06
4,block 1 GEYLANG,82.290323,428180.333312,4.703110e+03,4.703110e+03,POINT (103.88706 1.31819),4.229202e+02
...,...,...,...,...,...,...,...
1967,block 944 TAMPINES,145.727273,684870.708333,1.131450e+04,1.131450e+04,POINT (103.93169 1.35824),3.404486e+03
1968,block 946 HOUGANG,112.750000,422688.676740,7.787478e+03,7.787478e+03,POINT (103.88324 1.35147),5.638358e+02
1969,block 950 HOUGANG,110.875000,432444.543651,7.787478e+03,7.787478e+03,POINT (103.88324 1.35147),5.638358e+02
1970,block 954 HOUGANG,112.111111,423138.606934,7.787478e+03,7.787478e+03,POINT (103.88324 1.35147),5.638358e+02


Following this distances to parks in Singapore are considered.

In [19]:
parks = gpd.read_file('Data/Singapore Data/Singapore POI data/parks/parks-geojson.geojson')
parks

,Name,Description,geometry
0,kml_1,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.96086 1.34618 0.00000)
1,kml_2,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.97898 1.38990 0.00000)
2,kml_3,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.92313 1.40991 0.00000)
3,kml_4,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.86697 1.37924 0.00000)
4,kml_5,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.82907 1.37276 0.00000)
...,...,...,...
345,kml_346,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.89036 1.36796 0.00000)
346,kml_347,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.98919 1.39017 0.00000)
347,kml_348,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.88595 1.35520 0.00000)
348,kml_349,<center><table><tr><th colspan='2' align='cent...,POINT Z (103.90374 1.39164 0.00000)


The distance of a housing block centroid to a it's closest park is added to the dataframe below.

In [20]:
CalculateDistance(parks, 'Closest distance to a park')
singapore_2000_onwards

,ID,floor_area_sqm,mean_resale_price,Closest distance to Marina Bay Sands,Closest distance to Resorts World Casino Sentosa,geometry,Closest distance to a hotel,Closest distance to a park
0,block 1 BEDOK,87.151515,338909.623855,9.057217e+03,9.057217e+03,POINT (103.93147 1.32567),1.853188e+03,7.996054e+02
1,block 1 BUKIT MERAH,84.145833,417188.365358,2.915024e+03,2.915024e+03,POINT (103.83535 1.27935),4.987472e+02,5.739934e+02
2,block 1 BUKIT TIMAH,117.789474,698816.998405,2.635760e+05,2.635760e+05,POINT (101.46708 1.51128),2.536406e+05,2.484460e+05
3,block 1 CENTRAL AREA,62.666667,419808.253968,6.335036e+06,6.335036e+06,POINT (146.00830 -41.10814),6.332915e+06,6.330579e+06
4,block 1 GEYLANG,82.290323,428180.333312,4.703110e+03,4.703110e+03,POINT (103.88706 1.31819),4.229202e+02,4.021314e+02
...,...,...,...,...,...,...,...,...
1967,block 944 TAMPINES,145.727273,684870.708333,1.131450e+04,1.131450e+04,POINT (103.93169 1.35824),3.404486e+03,1.427613e+03
1968,block 946 HOUGANG,112.750000,422688.676740,7.787478e+03,7.787478e+03,POINT (103.88324 1.35147),5.638358e+02,2.796723e+02
1969,block 950 HOUGANG,110.875000,432444.543651,7.787478e+03,7.787478e+03,POINT (103.88324 1.35147),5.638358e+02,2.796723e+02
1970,block 954 HOUGANG,112.111111,423138.606934,7.787478e+03,7.787478e+03,POINT (103.88324 1.35147),5.638358e+02,2.796723e+02


Finally concentrations of traffic lights and parking lots within 500 meters of a housing block centroid in Singapore is added to the dataframe by the use of the following method which determines the point of interest count around a specific point and up until a specific distance from that point.

In [21]:
def determinePOICount(attribute, tag, threshold):
    counts = [];

    for i, row in singapore_2000_onwards.iterrows():
        count = 0
        for j, location in attribute.iterrows():
            coords1 = (singapore_2000_onwards.at[i, 'geometry'].y, singapore_2000_onwards.at[i, 'geometry'].x)
            coords2 = (attribute.at[j, 'geometry'].y, attribute.at[j, 'geometry'].x)

            distance = geopy.distance.geodesic(coords1, coords2).m
            if(distance < threshold):
                count += 1

        counts.append(count)

    singapore_2000_onwards[tag] = counts

In [22]:
traffic_lights= gpd.read_file('Data/Singapore Data/Singapore POI data/singapore-police-force-digital-traffic-red-light-cameras-shp/SPF_DTRLS.shp')
traffic_lights = traffic_lights[['ID', 'geometry']]
traffic_lights = traffic_lights.to_crs({'init': 'epsg:4326'})
traffic_lights

c:\Users\ankur\AppData\Roaming\jupyterlab-desktop\jlab_server\lib\site-packages\pyproj\crs\crs.py:130: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


,ID,geometry
0,90.0,POINT (103.89426 1.33571)
1,91.0,POINT (103.71576 1.34569)
2,92.0,POINT (103.90627 1.38878)
3,93.0,POINT (103.93599 1.35594)
4,94.0,POINT (103.82503 1.28649)
...,...,...
235,185.0,POINT (103.62716 1.30941)
236,186.0,POINT (103.89269 1.37661)
237,187.0,POINT (103.89962 1.40246)
238,188.0,POINT (103.86929 1.39161)


In [23]:
determinePOICount(traffic_lights, 'Traffic lights within 500 meters', 500)

In [25]:
parking_lots = gpd.read_file('Data/Singapore Data/Singapore POI data/ura-parking-lot/ura-parking-lot-geojson.geojson')

parking_lots['x'] = parking_lots.centroid.x
parking_lots['y'] = parking_lots.centroid.y

parking_lots = parking_lots [['Name', 'x', 'y']]

parking_lots = gpd.GeoDataFrame(parking_lots,
                             geometry=gpd.points_from_xy(parking_lots.x, parking_lots.y))

parking_lots = parking_lots[['Name', 'geometry']]
parking_lots

C:\Users\ankur\AppData\Local\Temp\ipykernel_16644\130702347.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parking_lots['x'] = parking_lots.centroid.x
C:\Users\ankur\AppData\Local\Temp\ipykernel_16644\130702347.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  parking_lots['y'] = parking_lots.centroid.y


,Name,geometry
0,kml_1,POINT (103.94655 1.33216)
1,kml_2,POINT (103.94657 1.33215)
2,kml_3,POINT (103.94667 1.33213)
3,kml_4,POINT (103.94652 1.33216)
4,kml_5,POINT (103.94659 1.33215)
...,...,...
42138,kml_42140,POINT (103.75038 1.32737)
42139,kml_42141,POINT (103.75056 1.32752)
42140,kml_42142,POINT (103.75162 1.32610)
42141,kml_42143,POINT (103.75159 1.32615)


In [26]:
determinePOICount(parking_lots, 'Parking lots within 500 meters', 500)
singapore_2000_onwards

,ID,floor_area_sqm,mean_resale_price,Closest distance to Marina Bay Sands,Closest distance to Resorts World Casino Sentosa,geometry,Closest distance to a hotel,Closest distance to a park,Traffic lights within 500 meters,Parking lots within 500 meters
0,block 1 BEDOK,87.151515,338909.623855,9.057217e+03,9.057217e+03,POINT (103.93147 1.32567),1.853188e+03,7.996054e+02,1,0
1,block 1 BUKIT MERAH,84.145833,417188.365358,2.915024e+03,2.915024e+03,POINT (103.83535 1.27935),4.987472e+02,5.739934e+02,0,300
2,block 1 BUKIT TIMAH,117.789474,698816.998405,2.635760e+05,2.635760e+05,POINT (101.46708 1.51128),2.536406e+05,2.484460e+05,0,0
3,block 1 CENTRAL AREA,62.666667,419808.253968,6.335036e+06,6.335036e+06,POINT (146.00830 -41.10814),6.332915e+06,6.330579e+06,0,0
4,block 1 GEYLANG,82.290323,428180.333312,4.703110e+03,4.703110e+03,POINT (103.88706 1.31819),4.229202e+02,4.021314e+02,1,412
...,...,...,...,...,...,...,...,...,...,...
1967,block 944 TAMPINES,145.727273,684870.708333,1.131450e+04,1.131450e+04,POINT (103.93169 1.35824),3.404486e+03,1.427613e+03,0,588
1968,block 946 HOUGANG,112.750000,422688.676740,7.787478e+03,7.787478e+03,POINT (103.88324 1.35147),5.638358e+02,2.796723e+02,0,9
1969,block 950 HOUGANG,110.875000,432444.543651,7.787478e+03,7.787478e+03,POINT (103.88324 1.35147),5.638358e+02,2.796723e+02,0,9
1970,block 954 HOUGANG,112.111111,423138.606934,7.787478e+03,7.787478e+03,POINT (103.88324 1.35147),5.638358e+02,2.796723e+02,0,9


The data is then exported below and is used by "Singapore housing analysis [Ankur Shanker].ipynb.

In [27]:
singapore_2000_onwards.to_csv('Singapore housing.csv')